In [8]:
# example of calculating the frechet inception distance in Keras for cifar10
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import shuffle
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
from keras.datasets import cifar10
import os
import cv2
import numpy as np

In [5]:
# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)

# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

In [11]:
def get_images(dir):
  # Get all JPEG images in the directory
  images_filenames = [filename for filename in os.listdir(dir) if filename.endswith(".png")]

  # Initialize an empty numpy array to hold the images
  images_array = np.zeros((len(images_filenames), 256, 256, 3), dtype=np.uint8)

  # Load each image and add it to the array
  for i, filename in enumerate(images_filenames):
    image = cv2.imread(os.path.join(dir, filename))
    image_resized = cv2.resize(image, (256, 256))
    images_array[i] = image_resized
      
  return images_array

In [12]:
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(256,256,3))
# load cifar10 images
# (images1, _), (images2, _) = cifar10.load_data()
real_image_path = '/vol/bitbucket/xz1919/luna-16-real-images/nodule_images'
diffusion_image_path = '/vol/bitbucket/xz1919/diffusion-generated-images/with-nodule/RESULTS/ADE20K-SDM-256CH/samples'
gan_image_path = '/vol/bitbucket/xz1919/GAN-images-with-nodule/samples'

images1 = get_images(real_image_path)
shuffle(images1)
images1 = images1[:1000]
images2 = get_images(diffusion_image_path)
print('Loaded', images1.shape, images2.shape)
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')
# resize images
images1 = scale_images(images1, (256,256,3))
images2 = scale_images(images2, (256,256,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
# calculate fid
fid = calculate_fid(model, images1, images2)
print('FID: %.3f' % fid)

Loaded (1000, 256, 256, 3) (1092, 256, 256, 3)
Scaled (1000, 256, 256, 3) (1092, 256, 256, 3)


2023-04-04 22:37:32.106275: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 786432000 exceeds 10% of free system memory.


32/32 [==============================] - 12s 355ms/step


2023-04-04 22:37:44.817139: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 858783744 exceeds 10% of free system memory.


35/35 [==============================] - 13s 361ms/step
FID: 80.820


In [13]:
images3 = get_images(gan_image_path)
print('Loaded', images1.shape, images3.shape)
# convert integer to floating point values
images3 = images3.astype('float32')
# resize images
images3 = scale_images(images3, (256,256,3))
print('Scaled', images1.shape, images3.shape)
# pre-process images
images3 = preprocess_input(images3)
# calculate fid
fid = calculate_fid(model, images1, images3)
print('FID: %.3f' % fid)

Loaded (1000, 256, 256, 3) (1024, 256, 256, 3)
Scaled (1000, 256, 256, 3) (1024, 256, 256, 3)


2023-04-04 22:40:02.423265: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 786432000 exceeds 10% of free system memory.


32/32 [==============================] - 11s 357ms/step


2023-04-04 22:40:14.174263: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 805306368 exceeds 10% of free system memory.


32/32 [==============================] - 12s 366ms/step
FID: 186.609


In [15]:
real_healthy_image_path = '/vol/bitbucket/xz1919/luna-16-real-images/images'
diffusion_healthy_image_path = '/vol/bitbucket/xz1919/diffusion-generated-images/without-nodule/ADE20K-SDM-256CH/samples'
gan_healthy_image_path = '/vol/bitbucket/xz1919/GAN-images-without-nodule'

images1 = get_images(real_healthy_image_path)
shuffle(images1)
images1 = images1[:5000]
images2 = get_images(diffusion_healthy_image_path)
print('Loaded', images1.shape, images2.shape)
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')
# resize images
images1 = scale_images(images1, (256,256,3))
images2 = scale_images(images2, (256,256,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
# calculate fid
fid = calculate_fid(model, images1, images2)
print('FID: %.3f' % fid)

Loaded (5000, 256, 256, 3) (1010, 256, 256, 3)
Scaled (5000, 256, 256, 3) (1010, 256, 256, 3)


2023-04-04 23:28:02.914833: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3932160000 exceeds 10% of free system memory.


32/32 [==============================] - 12s 373ms/step
FID: 84.494


In [16]:
images3 = get_images(gan_healthy_image_path)
print('Loaded', images1.shape, images3.shape)
# convert integer to floating point values
images3 = images3.astype('float32')
# resize images
images3 = scale_images(images3, (256,256,3))
print('Scaled', images1.shape, images3.shape)
# pre-process images
images3 = preprocess_input(images3)
# calculate fid
fid = calculate_fid(model, images1, images3)
print('FID: %.3f' % fid)

Loaded (5000, 256, 256, 3) (1024, 256, 256, 3)
Scaled (5000, 256, 256, 3) (1024, 256, 256, 3)
32/32 [==============================] - 12s 388ms/step
FID: 147.451
